Leveraging the power of CNN's to detect brain tumors in images of CT scans

In [ ]:
!pip install opendatasets

In [ ]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.optimizers import Adam, Lion, Adagrad
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Resizing, Rescaling, Dense, Dropout, Activation, Input, Flatten, AveragePooling2D
from keras.layers import RandomRotation, RandomZoom
from keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping
from keras.models import save_model
import opendatasets as od
import os
from PIL import Image
import matplotlib.pyplot as plt

Download the dataset

In [ ]:
od.download("https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: adityac001
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri


 Preprocess the data

In [ ]:
dataset_path = "/content/brain-tumor-classification-mri"
directories_dict = {}
files_dict = {}
labels = []
for dirs in os.listdir(dataset_path):
  dir_list = []
  for roots, directories, files in os.walk(dataset_path+r"/"+dirs):
    if len(roots)>0 and len(files)>0:
      dir_list.append(roots)
      labels.append(roots)
      files_dict[roots] = files
  directories_dict[dirs] = dir_list

print(files_dict)

In [ ]:
training_image_ds = keras.preprocessing.image_dataset_from_directory("/content/brain-tumor-classification-mri/Training",
                                                                        label_mode = "categorical",
                                                                        shuffle=True,
                                                                        seed=42,
                                                                        )

Classes/labels

In [ ]:
labels = training_image_ds.class_names
print(labels)

In [ ]:
testing_image_ds = keras.preprocessing.image_dataset_from_directory("/content/brain-tumor-classification-mri/Testing",
                                                                       label_mode = "categorical",
                                                                       )

Viewing the images

In [ ]:
def show_image():
  global files_dict
  roots = list(files_dict.keys())
  random_index = np.random.randint(0, len(roots)-1)
  files = os.listdir(roots[random_index])
  random_file_index = np.random.randint(0, len(files)-1)
  img = Image.open(roots[random_index]+"/"+files[random_file_index])
  img = np.asarray(img)
  plt.title(files[random_file_index])
  plt.imshow(img)
  print(img.shape)

show_image()

## Create the Model :
The model consists of image preprocessing layers followed by 2 blocks of **[Conv, Act, MaxPool]** and having a 3 layer deep dense neural network at its end with a dropout layer in between.

In [ ]:
model = Sequential(
    [
    Resizing(150, 150),
    Rescaling(1./255),
    Conv2D(filters=32, kernel_size=(3,3)),
    Activation('relu'),
    MaxPool2D(pool_size=2),
    Conv2D(filters=64, kernel_size=(2,2)),
    Activation('relu'),
    MaxPool2D(pool_size=2),
    BatchNormalization(),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(4, activation="softmax")
    ]
)

Instantiating an early stopping class

In [ ]:
es = EarlyStopping(patience=5, monitor='accuracy', restore_best_weights=True)

Compile the Model

In [ ]:
model.compile(optimizer = Adam(), loss = CategoricalCrossentropy(), metrics = ['accuracy'])

In [ ]:
history = model.fit(training_image_ds,  epochs=10, callbacks=[es])

In [ ]:
val = model.evaluate(testing_image_ds)

In [ ]:
import pandas as pd
df = pd.DataFrame(history.history['accuracy'])
df.plot()

Save the model

In [ ]:
model.save('/content/Model/braintumor_model.keras')

In [ ]:
model.save('/content/Model/braintumor_cnn.h5')

In [ ]:
!tar -czvf model.tar.gz /content/Model/braintumor_cnn.h5

In [ ]:
from google.colab import files
files.download('model.tar.gz')

In [ ]:
model.summary()